# Úvod

- PCA - Principal Component Analysis neboli analýza hlavních komponent je metoda lineární transformace dat
- Využití má především pro:
    * Kompresi dat
    * Redukci dimenze
    * Vizualizaci dat
    * Rozpoznávání
- V této prezentaci se zaměříme na redukci dimenze a vizualizaci

# Data

- Postup výpočtu budeme demonstrovat na databázi studentského hodnocení předmětů.
- Tato databáze se skládá z 28 specifických otázek a 5 doplňujících atributech.
- Popis otázek a atributů a samotnou databázi najdete zde: [@GunduzFokoue:2013]

# Popis postupu

1. příprava dat
2. korelační matice
3. spektrální rozklad + zobrazení vlivu jednotlivých komponent
5. vizualizace dat 
[@RPubs]

# Příprava dat

## Knihovny
- nejprve si připravíme všechny potřebné knihovny


In [ ]:
#| code-fold: false

using Pkg

Pkg.add("CSV") # načtení databáze
Pkg.add("DataFrames") # práce s databází
Pkg.add("LinearAlgebra") # výpočet spektrálního rozkladu
Pkg.add("Statistics") # výpočet kovarianční matice
Pkg.add("Printf") # stylování výpisů
Pkg.add("Plots") # vizualizace


using CSV
using DataFrames
using LinearAlgebra
using Statistics
using Printf
using Plots

include("print_funcs.jl"); # funkce na výpis vlivu jednotlivých komponent

## Načtení dat a kontrola

::: txt2
- Abychom mohli provést výpočet PCA, musíme ověřit, že nám nechybí žádná data a všechny prvky jsou čísla.
- Naše data mají 33 sloupců (otázek) a 5820 řádků (studentů)
:::


In [ ]:
# path to our dataset (.csv file)
file_path = joinpath(pwd(), "data","turkiye-student-evaluation_generic.csv")

# read data as a DataFrame
df = CSV.read(file_path, header=true, DataFrame)

# select the names of the columns for future use
column_names = names(df)

# we check that there are no missing values
if all(ismissing, eachcol(df))
    error("DataFrame have some Missing values!")
else 
    println("No value is missing.\n")
end

# we check that we are using only numerical values
if !all(t <: Number for t in eltype.(eachcol(df)))
    error("DataFrame has non-Numeric values!")
else
    println("All values are numeric.\n")
end

println(df[1:3, 1:7],"\n")
println(df[1:3, 8:16],"\n")
println(df[1:3, 17:25],"\n")
println(df[1:3, 26:end],"\n")

# Korelační matice

## Myšlenka

::: txt

- Jak můžeme vidět z výpisu, jednotlivá hodnocení studentů máme jako řádkové věktory. Proto musíme nejprve data transponovat, abychom měli matici v požadovaném tvaru: 
$$A=(a_1,\dots,a_{5820})$$

- Data následné musíme vycentrovat a standardizovat, abychom zajistili přesnější výpočet.

$$ \bar{a} = \frac{a_1 + a_2 + \dots + a_n}{n} $$

$$ \sigma = \sqrt{\frac{1}{n - 1} \sum_{i=1}^{n} (a_i - \bar{a})^2} $$

::: pozn
Pozn: mocnění v sumě probíhá po složkách jednotlivě
:::

$$ a_{i_{new}} = \frac{a_i - \bar{a}}{\sigma_i}$$

- Nyní můžeme vypočítat korelační matici otázek

$$ C = AA^T$$

:::

## Výpočet


In [ ]:
# we center and scale our data and create a covariance matrix
data_matrix = Matrix(df)'
N, M = size(data_matrix)
centered_scaled_data = (data_matrix .- mean(data_matrix, dims=2)) ./ std(data_matrix, dims=2)
cov_data = centered_scaled_data * centered_scaled_data' / (M-1)
df_cov = DataFrame(cov_data, column_names)
insertcols!(df_cov, 1, :Names => column_names)

show(df_cov)

# Spektrální rozklad

## Myšlenka

- Jakmile máme kovarinační matici, která je symetrická a semidefinitní, můžeme vypočítat spektrální rozklad

$$ C = AA^T = V \Lambda V^T $$

- funkce eigen() z balíčku LinearAlgebra nám vrátí vlastní čísla a jejich příslušné vektory ve vzestupném pořadí
- Pro lepší náhled si je přerovnáme do sestupného pořadí

## Výpočet


In [ ]:
# calculate eigen values and eigen vectors
λ, V = eigen(cov_data);
# reverse them so the eigen values are in descending order
λ = reverse(λ);
V = reverse(V, dims = 2);

@printf("Size of lambda: %d\n", size(λ)[1])
@printf("Size of V: %d %d\n", size(V)[1], size(V)[2])
println("Eigen values:")
println(λ)

## Chyba jednotlivých dimenzí

::: txt2

- Pro každou komponentu (dimenzi) nově vzniklé matice můžeme určit procentuální hodnotu, jak dobře aproximuje původní prostor
- Lze vidět, že už s využitím pouze 5 prvních komponent (tedy 5 dimenzí) máme chybu pouze 15%

:::


In [ ]:
print_info(λ)

# variance.percent(dim = i) = λi / sum(λ)

## Vliv jednotlivých komponent


In [ ]:
print_components(λ)

# Vizualizace dat

## Studenti - graf


In [ ]:
dim2V = V[:, 1:2]

t = dim2V' * centered_scaled_data

plotly()
scatter(t[1, :], t[2, :], legend=nothing)

## Studenti

- Na předchozím obrázku můžeme vidět vizualizaci odpovědí jednotlivých studentů ve 2 dimenzích
- Všimněme si, že v datech se objevuje 5 shluků (clusterů)
- Pokud si vypíšeme původní hodnoty odpovědí pro studenty z každého clusteru, zjistíme, že téměř všechny odpovědi byly vždy stejné
- V prvním clusteru (nejvíce vpravo) mají odpovědi na otázky Q1-Q28 hodnotu 1, pro druhý 2 atd.
- Body si můžeme obarvit podle průměru odpovědí na tyto otázky


## Studenti - obarveno


In [ ]:
function get_value(x)
    if isapprox(x, 1, atol=0.1)
        return 1
    elseif isapprox(x, 2, atol=0.1)
        return 2
    elseif isapprox(x, 3, atol=0.1)
        return 3
    elseif isapprox(x, 4, atol=0.1)
        return 4
    elseif isapprox(x, 5, atol=0.1)
        return 5
    else
        return 6
    end
end

mean_of_Q_answers = mean(data_matrix[6:end, :], dims=1)'
vals = [get_value(x) for x in mean_of_Q_answers]
indices_1 = findall(x -> x == 1, vals)
indices_2 = findall(x -> x == 2 , vals)
indices_3 = findall(x -> x == 3 , vals)
indices_4 = findall(x -> x == 4, vals)
indices_5 = findall(x -> x == 5 , vals)
indices_other = findall(x -> x == 6, vals)
scatter(t[1, indices_1], t[2, indices_1],color=:red, label="1")
scatter!(t[1, indices_2], t[2,indices_2], color=:purple, label="2")
scatter!(t[1, indices_3], t[2,indices_3], color=:green, label="3")
scatter!(t[1, indices_4], t[2,indices_4], color=:yellow, label="4")
scatter!(t[1, indices_5], t[2,indices_5], color=:orange, label="5")
scatter!(t[1, indices_other], t[2,indices_other], color=:blue, label="other")

## Studenti 3.dim

In [ ]:
dim3V = V[:, 1:3]

d = dim3V' * centered_scaled_data

scatter(d[1, :], d[2, :], d[3, :], legend=nothing, ms=1)

## Otázky

- Pro vizualizaci vlivu jednotlivých otázek na jednotlivé komponenty musíme vytvořit korelační matici jednotlivých studentů
- Místo $AA^T$ uděláme:
$$ A^TA $$

## Otázky - graf


In [ ]:
cov_data2 = centered_scaled_data' * centered_scaled_data / (M-1);

λ2, V2 = eigen(cov_data2);
# reverse them so the eigen values are in descending order
λ2 = reverse(λ2);
V2 = reverse(V2, dims = 2);

dim2V2 = V2[:, 1:2];
t2 = centered_scaled_data * dim2V2;
size(t2)
x = t2[:, 1]';
y = t2[:, 2]';
scatter(x, y, legend=nothing)
annotate!([(xi, yi, text(name, :bottom, 8)) for (xi, yi, name) in zip(x, y, column_names)])

# Závěr
- PCA je poměrně jednoduchá metoda pro práci i s velkými daty a může nám pomoct porozumět rozložení dat nebo třeba ke klasifikaci dat (v případě strojového učení)

- Zde se nám povedlo promítnout body do prvních 2 dimenzí s chybou 25%, a do prvních 3 s chybou 22%

- Pro klasifikaci bychom mohli použít prvních 8 dimenzí s chybou už pouhých 10%

# Bibliografie

::: {#refs}
:::